<tip: for better readability you can press ctrl or cmd key along with plus sign key for zooming in>

# Parallel Painting - Python explanation of a CUDA parallel painting algorithm

## Welcome!

Welcome to my first article!<br>
It is about a parallel algorithm I came up with some years ago and implemented in CUDA, executed on an NVIDIA GTX card.
It was the heart of a post process algorithm used for a neural network output.
As you will see, this algorithm in itself is very interesting and pretty spectacular for the first sight.


### Color islands
<img align="Right" src="./pictures/color_islands.png" alt='colorful islands in a light blue sea' width = 40% style = "padding-left: 16px">

    
The image on the right hopefully looks like some fancy colored islands in a calm sea, at least that was the goal when I tried to use my finger-painting skills on a small smartphone touchscreen.
I could draw black island outlines on a white screen with a simple application. The app had a paint function, which showed a tilted paint bucket filled with paint. I could move the bucket around with my finger, and also "pour out its content" with a button on the bottom of the screen. The application flooded all the white parts with light blue when I poured the carefully selected color for the sea outside of all island contours. Not yet knowing the colors of the islands, I moved the bucket inside all of the islands and poured black paint into all of them. This way I was also able to remove the outlines themselves as the islands turned all black.

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;

<img align="Left" src="./pictures/black_islands.png" width = 40% style = "padding-right: 16px">
<br>
I can not say I had a clear plan about the colors of the islands, but started to move around the bucket, set the color of its paint and tapped the 'pour out' button when the bucket was over a black area. The application relentlessly turned the black islands into color ones.
The painting process looked instantaneous, the paint flooded an island in a blink of an eye. It is quite convenient from the user's point of view, however, let us be honest, it is not quite realistic. At least all paint or liquid I've ever seen was flowing way much slower than the speed of light. Hopefully I didn't miss something and this is always the case.
I guess you wouldn't be so surprised if I was telling that the "paint" in the application is also flowing in some direction or directions, flooding everything until it hits the sea, which acts as a wall. It either stops the paint in that certain direction, or more probably redirects it to other black areas, if anymore is left at all in the neighborhood. 
Sounds pretty complex! Thankfully or sadly even small devices like smartphones are fast enough to do this complex task immediately.
If you are courius about how this painting process works under the hood, you are just like me! If not, further reading of this article can bring up your interest.
I don't know how the particular paint filling in the app works, not least because it paints instantly, but I have hazy memories from my childhood, when things were slower, especially computers. You could easily follow the painting process, sometimes for long tens of seconds. If I remember correctly they more looked like brush strokes. Pixels were drawn up then down between the outlines, stepping one pixel to the sides after each vertical stroke.
What I'm completely sure about that pixels were painted one by one, only one pixel was drawn at a time. It is completely understandable, as processors could do only one thing at a time, they had single cores only. Nevertheless, designing an algorithm for one core is usually simpler. How could 8 or 16 cores help us out here? Can a brush stroke draw 8 pixels at once? What if one of the pixels hits the outline, how would the other pixels know that they should not be drawn as they would fall outside? Similarily, could we have 16 brush strokes at once? These are some of the questions came to my mind. Well, I'm sure there are clever answers to them, but I'm afraid they are rather complicated answers. So I would recommend us to stick ourselves to a one-step-at-a-time method when we are thinking about a simple solution.
Now I can hear you asking why I'm talking here about a simple one thread (one core) solution, when you arleady read the title and preface of this article. Spoiler alert: yes, I will introduce my paralell algorithm later on in this article, wich runs on thousands of cores of a GPU at once (!), manipulating thousands of pixels at a time! That will be another story, will look more like a flood fill, for now, let me introduce first a 'classic', one pixel at a time, brush stroke like algorithm to see the contrast between the two.

### A single core painting algorithm

Thinking back to old brushstroke algorithms, I don't remember exactly how an entire shape was filled with pixels. Just imagine, we specify the starting point inside a rectangle shape, pixels starts to appear upwards with a given color, then downwards one pixel to the left, then up again to the left again until we reach the left side of the shape. But what about the other side, the right one from the starting point? If I remember correctly, after the left side was painted, pixels started to appear downwards from the starting point, then upwards one pixel to the right, all the way to the right side. It was a kind of 'restart' from the starting point. Now you can imagine what is it like having a more complex shape. Such as a rotated or lying **H** shape, one thin vertical rectangle with two thin horizontal rectangles on its bottom and top. That must have like 3 'restarts' from 3 different pixels. It doesn't sound as a simple algorithm, it must be quite complex!<br>
I was already thinking about this process for this article, when I asked my lazy programmer self about his (my) opinion. It is indeed complex, but let's outsource the complex 'restart' part of the method to the machine!
Let's start with easier things, moreover, let's start in the very beginning! We are given a pixel coordinate to start somewhere inside a black island on an image, and also a color so that we know which color to use for drawing pixels.
We could write this Python function header:
```python
def paint(image, x, y, color):
```
Can we simply paint the given pixel with the specified color in the very first step? Surely, but we need to be cautious. If the pixel at the given coordinates is not black for some reason, because it is a blue sea pixel or an already painted island pixel, we should not paint it. Also, the painting process itself should not be started. In this case, we will simply return from the paint function. Otherwise, we can paint the pixel to the given color.<br>
If you feel like something is missing, an error message for the fail case or the sanity check of the x and y coordinates, your intuition is right! However, I would skip the sanity check for simplicity and I should skip the error message for reasons you are about to see.
So, if the given coordinates point to a pixel inside a black island, we can paint the first pixel. So far, so good! But what about the others? Should we go up one pixel, down, or what? Now let's see the simple approach of my lazy programmer self.
Where are the remaining black pixels? Around the pixel we have alredy painted! Great situational awareness, but what can we do with this information? Let's say, we never go too far, inspecting only the pixels around the already painted pixel. Ideally they are all black, or at least most of them. Let's try to paint all the neighbors around the painted pixel, one by one. We take one neighbor by its coordinates and if it is not black (not already painted or sea pixel), we don't do anything with it. If it is black, we paint it to the given color.
Sounds familiar? Yes, we have just done the same with the first pixel! And here comes the trick: after we paint a pixel, we always take all its neighbors again and do the exact same with those pixels one by one. If possible, paint it to the given color and take its neighbors one by one again.
I know it sounds ridiculous, mostly because one of the neighbors of a pixel neighbor is the pixel itself, which is already painted (hope I haven't lost you at this point and you are still with me). It is not a problem as we always need to check every pixel before painting anyway.
As the task is the same for the first pixel and all the pixels around it, we can simply call the paint function itself inside the paint function, for all neighboring pixels. Calling a function inside that particular function may sound like a recipe for a trouble, coming in the form of a nice infinite loop, and it certainly can be if a function always calls itself. However, we already have a case when we don't need to inspect the neighbors. It is when the pixel we are working on is an already painted or a sea pixel. In that case we return from the function without calling it again.

Sounds good, let's see the whole function in Python! 

```python
def paint(image, x, y, color):
    if image[y,x] == black_color:
        image[y,x] = color
        paint(image, x    , y - 1, color) # neighbor above
        paint(image, x    , y + 1, color) # below
        paint(image, x - 1, y    , color) # to the left
        paint(image, x + 1, y    , color) # to the right
        paint(image, x - 1, y - 1, color) # top left neighbor
        paint(image, x + 1, y - 1, color) # top right
        paint(image, x - 1, y + 1, color) # bottom left
        paint(image, x + 1, y + 1, color) # bottom right
    return
```
> **_Note:_**
>- ```image``` is a 2D numpy array of color code numbers, the image of islands
>- ```color``` is a color code number between 0 and 11, 0 means black, 1 is the sea color, numbers between 2 and 11 represent island colors
>- ```black_color``` is 0
>- as usual, x is the horizontal axis, y is the vertical
>- indexing a 2D numpy array is a bit tricky, y comes first, x is the second
>- numpy arrays are mutable, which means any modification of the array inside the function is also seen from outside (passed by reference in C++ terms)



You can see 8 self-function calls inside the function as all pixels have 8 neighbors around.
Smart people have already named these kind of self-calling functions as recursive functions, because they recursively call themselves most of the time. Don't worry if you are not familiar with this concept, and don't immediately trust anyone who claims to know how and when the calls and returns occur in a small but complex recursive function. Often, as in our case, this is not known in advance, since it depends on what data the function is working on. In our case, this data is the shape of the island and the location of the starting point. The path of function calls and returns is automatically handled by the machine. With recursion, the only thing we need to care about is the maximum depth, that is, the number of times the function calls itself before a return happens. In our case, it can be a few thousand, which is more than the thousand recursion depth allowed by default in Python.<br>We can easily change this upper limit:

```python
import sys
sys.setrecursionlimit(4000)
```

Let's see how it works, I mean literally see it!

<img align="Right" src="./pictures/color_islands.png" alt='colorful islands in a light blue sea' width = 40% style = "padding-left: 16px">





That may sound like a recipe for trouble, and it certainly can be if we are not careful. <- quote from Barron Stone
$$ O(n^2) $$
$$ O(log_2(n)) $$
$$ \mathcal{O}(log_2n) $$


Type Markdown and LaTeX:  𝛼{2}